# Data cleaning

In [33]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [5]:
%env GCLOUD_PROJECT=reddit-254019

env: GCLOUD_PROJECT=reddit-254019


## Comments

In [0]:
%%bigquery --project reddit-254019 df_comments
WITH
  q1 AS(
  SELECT
    subreddit,
    body
  FROM
    `fh-bigquery.reddit_comments.2019_01`
  WHERE
    LENGTH(body) > 10
    AND body != '[deleted]'
    AND body != '[removed]'
    AND body != '[ Removed by reddit in response to a copyright notice. ]'
    AND body != 'NaN'
    AND body != '' ),
  q2 AS(
  SELECT
    subreddit,
    body
  FROM
    q1
  WHERE
    subreddit IN ('politics',
      'funny',
      'gaming',
      'nba',
      'worldnews',
      'todayilearned',
      'movies',
      'IAmA',
      'atheism',
      'aww',
      'technology',
      'Fitness',
      'europe',
      'science'))
SELECT
  subreddit,
  body
FROM
  q2

In [35]:
df_comments.head()

,subreddit,body
0,aww,Sigh
1,aww,Just came here to say that we have exact same ...
2,aww,Unequivocally “no”
3,aww,Awesome hubby you have and great FEDEX driver....
4,aww,He is so handsome. When I was little my parent...


In [0]:
import pandas as pd
import re
import numpy as np

The DataSet has been named `df_comments` and it contain comments information of subreddit and body of the comment.

### Pre cleaning filters with SQL

The df has been pre "cleaned":
- no data with body "deleted"
- no data with body "removed"
- no data with body "Removed by reddit in response to a copyright notice."
- no NAN on body
- no empty data on body

### Comments - Data Overview

In [37]:
df_comments.shape

(8350392, 2)

In [38]:
df_comments.describe()

,subreddit,body
count,8350392,8350392
unique,14,7906417
top,politics,"\nAs a reminder, this subreddit [is for civil ..."
freq,2366194,30123


Check that there are not NaN values

In [39]:
df_comments.isnull().sum()

subreddit    0
body         0
dtype: int64

Checking the largest text in our comments


In [40]:
df_comments.body.map(lambda x: len(x)).max()

19983

In [41]:
df_comments.body.map(lambda x: len(x)).min()

1

### Comments -  Data Cleaning

#### HTML


As HTLM data does not add more informatio to our text, we clean it. Do to that, we define a function to clean data with regular expressions.

http: matches literal characters \S+: matches all non-whitespace characters (the end of the url) we replace with the empty string

In [0]:
def remove_html_tags(text):
    clean = re.compile(r'http\S+')
    return re.sub(clean, '', str(text))

In [0]:
df_comments['body'] = df_comments['body'].map(remove_html_tags)

#### Line endings


Removing **\n** text from comments

In [0]:
def remove_lineEndings (text):
    clean = re.compile(r'\n')
    return re.sub(clean, '', str(text))

In [0]:
df_comments['body'] = df_comments['body'].map(remove_lineEndings)

#### Digits/Punctuation/Symbols

Even if significant information can be hidden in the appearance of the symbols, you’d like to get rid of them.

In [0]:
def remove_symbols (text):
    clean = re.compile(r'[^a-zA-Z0-9 r/]') #r/ is how to call a subreddit, I want keep it
    return re.sub(clean, '', str(text))

In [0]:
df_comments['body'] = df_comments['body'].map(remove_symbols)

## Posts

In [0]:
%%bigquery --project reddit-254019 df_posts
WITH
  q1 AS (
  SELECT
    subreddit,
    title,
    selftext
  FROM
    `fh-bigquery.reddit_posts.2019_01`
  WHERE
    (LENGTH(title) > 5 AND LENGTH(selftext) > 5)
      or ((selftext != '[deleted]' AND length(title) > 5)
      AND (selftext != '[removed]' AND length(title) > 5)
      AND (selftext != '[ Removed by reddit in response to a copyright notice. ]' AND length(title) > 5)
      AND (selftext != 'NaN' AND length(title) > 5)
      AND (selftext != '' AND length(title) > 5))),
  q2 AS(
  SELECT
    subreddit,
    title,
    selftext
  FROM
    q1
  WHERE
    subreddit IN ('politics',
      'funny',
      'gaming',
      'nba',
      'worldnews',
      'todayilearned',
      'movies',
      'IAmA',
      'atheism',
      'aww',
      'technology',
      'Fitness',
      'europe',
      'science'))
SELECT
  subreddit,
  title, 
  selftext
FROM
  q2

In [21]:
df_posts.head()

,subreddit,title,selftext
0,nba,Injury Bug,"Out of all of the teams in the league, which o..."
1,nba,Who's the most likely Western Conference All s...,I predict it's going to be Rudy Gobert. He sho...
2,nba,Trying to compile a comprehensive list of the ...,Would really appreciate y’all’s favorites. I’m...
3,nba,"When talking about bad contracts, why does nob...",Otto Porter has max contract courtesy of the B...
4,nba,How did Steven Adams get more All Star votes f...,Karl is averaging 23/12/3 with 1.9 blocks and ...


The DataSet has been named `df_postss` and it contain comments information of subreddit title and selftext of a post.

### Pre cleaning filters with SQL

The df has been pre "cleaned":
- no data with selftext "deleted" and body length < 5
- no data with selftext "removed" and body length < 5
- no data with selftext "Removed by reddit in response to a copyright notice." and body length < 5
- no NAN on selftext and body length < 5
- no empty data on selftext and body length < 5

### Posts - Data Overview

In [22]:
df_posts.shape

(271117, 3)

In [23]:
df_posts.describe()

,subreddit,title,selftext
count,271117,271117,271117
unique,16,261789,13778
top,AskReddit,"Men, what do you find most confusing about women?",[removed]
freq,180033,82,141715


Check that there are not NaN values

In [24]:
df_posts.isnull().sum()

subreddit    0
title        0
selftext     0
dtype: int64

Checking the largest text in our comments

In [25]:
df_posts.title.map(lambda x: len(x)).max()

320

In [26]:
df_posts.selftext.map(lambda x: len(x)).max()

30407

### Posts -  Data Cleaning

#### HTLM

Removing __html__ text from posts using `remove_html_tags` function

In [0]:
df_posts['title'] = df_posts['title'].map(remove_html_tags)

In [0]:
df_posts['selftext'] = df_posts['selftext'].map(remove_html_tags)

#### Line endings

Removing **\n** text from posts using `remove_lineEndings`function

In [0]:
df_posts['title'] = df_posts['title'].map(remove_lineEndings)

In [0]:
df_posts['selftext'] = df_posts['selftext'].map(remove_lineEndings)

#### Digits/Punctuation/Symbols

Removing __symbols__ text from posts usign `remove_symbols`funtction

In [0]:
df_posts['title'] = df_posts['title'].map(remove_symbols)

In [0]:
df_posts['selftext'] = df_posts['selftext'].map(remove_symbols)